In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pysentiment2 as ps
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import yfinance as yf


def sentiment_vader(var):
    #pip install vaderSentiment
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
    sentiment = SentimentIntensityAnalyzer() 
    out_score = sentiment.polarity_scores(var)
    return out_score
def sentiment_ps(var):
    lm = ps.LM()
    tokens = lm.tokenize(var)
    score = lm.get_score(tokens)
    return score
def topic_distribution(string_input):
    string_input = [string_input]
    # Fit and transform
    X = vect.transform(string_input)
 
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
 
    output = list(ldamodel[corpus])[0]
 
    return output
def topic_prediction(my_document):
    string_input = [my_document]
    X = vect.transform(string_input)
    # Convert sparse matrix to gensim corpus.
    corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)
    output = list(ldamodel[corpus])[0]
    topics = sorted(output,key=lambda x:x[1],reverse=True)
    return topics[0][0]
def normalize(arr):
     return (arr-min(arr))/(max(arr)-min(arr))
def normalize_2(arr):
     return np.interp(arr, (arr.min(), arr.max()), (-1, +1))

In [2]:
df = pd.read_pickle("sentiment.pkl")
df['Year']= pd.to_datetime(df['Year'])
df = df.set_index('Year')
df['Year'] = df.index

In [9]:
df1 = df[df['Year'] > "1999-03-10"]

KeyError: 'Year'

Year
1993-02-03    False
1993-03-23    False
1993-05-18    False
1993-07-07    False
1993-08-17    False
1993-09-21    False
1993-11-16    False
1993-12-21    False
1994-02-04    False
1994-03-22    False
1994-05-17    False
1994-07-06    False
1994-08-16    False
1994-09-27    False
1994-11-15    False
1994-12-20    False
1995-02-01    False
1995-03-28    False
1995-05-23    False
1995-07-06    False
1995-08-22    False
1995-09-26    False
1995-11-15    False
1995-12-19    False
1996-01-30    False
1996-03-26    False
1996-05-21    False
1996-07-02    False
1996-08-20    False
1996-09-24    False
              ...  
2017-06-14     True
2017-07-26     True
2017-09-20     True
2017-11-01     True
2017-12-13     True
2018-01-31     True
2018-03-21     True
2018-05-02     True
2018-06-13     True
2018-08-01     True
2018-09-26     True
2018-11-08     True
2018-12-19     True
2019-01-30     True
2019-03-20     True
2019-05-01     True
2019-06-19     True
2019-07-31     True
2019-09-18     

In [3]:
# Get Stock Market Data: US 10-year T-bill
import yfinance as yf
stock_info = yf.download("QQQ", start="1993-02-01", end="2021-02-20")
stock_info.index = pd.to_datetime(stock_info.index)
stock_info

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-03-10,51.125000,51.156250,50.281250,51.062500,44.386864,5232000
1999-03-11,51.437500,51.734375,50.312500,51.312500,44.604156,9688600
1999-03-12,51.125000,51.156250,49.656250,50.062500,43.517586,8743600
1999-03-15,50.437500,51.562500,49.906250,51.500000,44.767155,6369000
1999-03-16,51.718750,52.156250,51.156250,51.937500,45.147457,4905800
1999-03-17,51.937500,52.000000,51.406250,51.562500,44.821491,3965000
1999-03-18,51.500000,52.593750,51.484375,52.562500,45.690742,4848400
1999-03-19,53.250000,53.250000,51.187500,51.218750,44.522667,7160400
1999-03-22,51.437500,51.562500,50.500000,50.593750,43.979385,5024800


In [4]:
# stock market volatibility within 10 days after the Fed meetings
import numpy as np
price_changes = []
price_change_percentages = []
labels = []
window = 5
# workaround for rolling widow
for index, row in df.iterrows():
    minutes_date = row['Year']
    try:
        start = stock_info.index.get_loc(minutes_date)
    except:
        start = stock_info.index.get_loc(minutes_date + pd.DateOffset(days=1))
    stock_closing_sum = stock_info.iloc[start+window]['Close']
    start_price = stock_info['Close'].iloc[start]
    price_change = stock_closing_sum - start_price
    price_change_percent = (stock_closing_sum - start_price) / start_price
    price_changes.append(price_change)
    price_change_percentages.append(price_change_percent)
    

df['price_change'] = price_changes
df['price_change_percent'] = price_change_percentages
df['label'] = df['price_change_percent'].apply(lambda x: 1 if x> df["price_change_percent"].quantile(0.75) or x < df["price_change_percent"].quantile(0.25) else 0)

KeyError: Timestamp('1993-02-04 00:00:00')

In [5]:
df

,Statement,polarity,positive,negative,subjectivity,Year
Year,,,,,,
1993-02-03,A meeting of the Federal Open Market Committee...,-0.134503,74,97,0.080546,1993-02-03
1993-03-23,A meeting of the Federal Open Market Committee...,-0.302857,61,114,0.080571,1993-03-23
1993-05-18,A meeting of the Federal Open Market Committee...,-0.259615,77,131,0.093990,1993-05-18
1993-07-07,A meeting of the Federal Open Market Committee...,-0.156627,35,48,0.038444,1993-07-07
1993-08-17,A meeting of the Federal Open Market Committee...,-0.224299,83,131,0.097717,1993-08-17
1993-09-21,A meeting of the Federal Open Market Committee...,-0.173913,76,108,0.084949,1993-09-21
1993-11-16,A meeting of the Federal Open Market Committee...,0.023810,86,82,0.076642,1993-11-16
1993-12-21,A meeting of the Federal Open Market Committee...,0.024876,103,98,0.089096,1993-12-21
1994-02-04,A meeting of the Federal Open Market Committee...,-0.227723,78,124,0.088830,1994-02-04


In [17]:
#Start feature dataset
X = df
X = np.array(X.drop(['label','Statement','Year','positive','negative','price_change','price_change_percent'],1))
y = np.array(df['label'])

In [18]:
#Split(80% training, 20% test)
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size= 0.4, random_state = 0)

In [19]:
# Create and train model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [20]:
predictions = model.predict(x_test)
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.56      0.47      0.51        43
           1       0.55      0.64      0.59        44

    accuracy                           0.55        87
   macro avg       0.55      0.55      0.55        87
weighted avg       0.55      0.55      0.55        87



In [21]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [22]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
m = randomclassifier.fit(x_train,y_train)
predictions_2 = m.predict(x_test)
score = accuracy_score(y_test,predictions)
report = classification_report(y_test,predictions)
print(score)
print(report)

0.5517241379310345
              precision    recall  f1-score   support

           0       0.56      0.47      0.51        43
           1       0.55      0.64      0.59        44

    accuracy                           0.55        87
   macro avg       0.55      0.55      0.55        87
weighted avg       0.55      0.55      0.55        87

